# Fake news generator

Verificar se apartir de um post de fack checking é possivel obter um gerador de fake news.

In [5]:
#Importar as bibliotecas necessarias para a leitura dos posts

import json
import pandas as pd
from matplotlib.font_manager import json_load

import readNews
import sentimentAnalysis

### Read the news

In [6]:
def remove_values_from_list(the_list, val):
    return [value for value in the_list if value != val]

def clean (ListSentences):
    
    #limpeza '
    l = [s.replace("\"","'") for s in ListSentences]
    
    #Separar em frases + pequenas
    sentences = []
    for i in l:
        if "Polígrafo" in i:
            continue
        sentences = sentences + i.split('.')

    return remove_values_from_list(sentences, "\'\'")

In [25]:



url = 'https://poligrafo.sapo.pt/fact-check/ucrania-entregou-armas-nucleares-a-russia-em-troca-da-garantia-de-nunca-ser-amecada-ou-invadida'
language = 'pt'
summary = readNews.main(url, "summary").encode('utf8').decode().split('\n')
title = readNews.main(url, "title").encode('utf8').decode().split('\n')
newsSentences = title + summary
text = title + readNews.main(url, "text").encode('utf8').decode().split('\n')



In [26]:
#newsSentences


In [27]:
#text

#### Sentiment Analysis

In [28]:

text =clean(text)

sentiments, status_code = sentimentAnalysis.main(text,"pt")


In [29]:
text

["Ucrânia entregou armas nucleares à Rússia 'em troca da garantia de nunca ser ameaçada ou invadida'?",
 "'Em 1994, a Ucrânia detinha um gigantesco arsenal nuclear até assinar o Memorando de Budapeste e entregou cerca de 1",
 "600 armas nucleares remanescentes da antiga União Soviética à Rússia, em troca de um tratado de paz e da garantia de nunca ser invadida ou ameaçada', lê-se no post de 24 de fevereiro no Facebook",
 '',
 '',
 "'Hoje, a Rússia invade a Ucrânia demonstrando o resultado de entregar a sua capacidade de defesa a alguém contando com a sua benevolência', acrescenta-se",
 '',
 '',
 'São factos históricos?',
 '',
 '',
 "'Os promotores desta iniciativa foram, de facto, os norte-americanos, que receavam o espalhamento do arsenal nuclear soviético em pequenas repúblicas recém criadas e politicamente instáveis",
 " Além disso, a enorme concentração de armas nucleares na Ucrânia era um entrave à expansão da NATO para o Leste', acrescenta",
 '',
 '',
 'Segundo o historiador, os 

In [40]:
status_code

200

In [41]:
## Ordenar por ordem decrescente maior emoçao
playload = json.loads(sentiments)

In [50]:

## Ordenar as frases por aquela que tem maior surpresa e tristeza
sortedByEmotion = {k: v for k, v in sorted(playload.items(), key=lambda item: (item[1]['emotions']['sadness'] + item[1]['emotions']['surprise']), reverse=True)}


vizualizar os valores de emoçao de surpresa e tristeza

In [70]:
list(sortedByEmotion.items())[0][1]['emotions']['surprise'] 

0.29

In [71]:
list(sortedByEmotion.items())[0][1]['emotions']['sadness']

0.14

### Save Data

In [72]:
data = pd.read_csv('news.csv')

In [73]:
row = {'url': url, "language": language, "text": list (sortedByEmotion) [0], "surprise": list(sortedByEmotion.items())[0][1]['emotions']['surprise'] , 'sadness':list(sortedByEmotion.items())[0][1]['emotions']['sadness']}

# Adicionar a nova linha ao dataset
data = data.append(row,  ignore_index=True)

#Retirar colunaas que estão a mais
data = data[['url','language','text', "surprise" ,'sadness' ]]


In [74]:
data

,url,language,text,surprise,sadness
0,https://poligrafo.sapo.pt/fact-check/ucrania-e...,pt,Questionado sobre se o presente ataque militar...,0.29,0.14


In [75]:
data.to_csv('news.csv')

### Automatizar o processo


        1) Extrair conteudo do url da noticia
        2) Divir o conteudo extraido por frases simples (split pelo .)
        3) Realizar analise de sentimentos nas frases;
        4) Identificar a frase com maios percentagem de tristeza e surpresa,
        5) Guardar resultados num csv.

In [11]:
 ### Recolher os links que se pretende analisar
    
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re

req = Request("https://poligrafo.sapo.pt/fact-checks")
html_page = urlopen(req)

soup = BeautifulSoup(html_page, "lxml")

links = []
for link in soup.findAll('a'):
    if type (link.get('href')) == str and (len( link.get('href'))) >  50:  
        links.append("https://poligrafo.sapo.pt" + link.get('href'))


In [1]:
list_urls = links[:-1]
list_urls

NameError: name 'links' is not defined

In [13]:
language = 'pt'

def extractor (url):
  #  summary = readNews.main(url, "summary").encode('utf8').decode().split('\n')
  #  title = readNews.main(url, "title").encode('utf8').decode().split('\n')
 #   newsSentences = title + summary
    text = readNews.main(url, "text").encode('utf8').decode().split('\n')
    return text 

def sentiment_analysis(text, language):  
    
    ## Limpar o texto, transformar paragrafos em frases
    text =clean(text)
    
    status_code = 0 ## Valor inicial 
    
    while (status_code != 200):
        emotions, status_code = sentimentAnalysis.main(text,language)
    playload_emotions = json.loads(emotions)
    
    return playload_emotions
    

    
def main ():
    for url in list_urls:
        text =  extractor (url)
    
        playload_emotions = sentiment_analysis(text, language)

        ## Ordenar as frases por aquela que tem maior percentagem de sadness e surprise
        sortedByEmotion = {k: v for k, v in sorted(playload_emotions.items(), key=lambda item: (item[1]['emotions']['sadness'] + item[1]['emotions']['surprise']), reverse=True)}
        print(sortedByEmotion)
        data = pd.read_csv('news.csv')
        
        row = {'url': url, "language": language, "text": list (sortedByEmotion) [0], "surprise": list(sortedByEmotion.items())[0][1]['emotions']['surprise'] , 'sadness':list(sortedByEmotion.items())[0][1]['emotions']['sadness']}
        print(row)
        # Adicionar a nova linha ao dataset
        data = data.append(row,  ignore_index=True)

        #Retirar colunaas que estão a mais
        data = data[['url','language','text', "surprise" ,'sadness' ]]
        
        data.to_csv('news.csv')

In [14]:
main()

{"Além disso, a GNU sublinha que o documento não é secreto: 'Na gestão de negócios secretos, há um procedimento de registo diferente": {'emotions': {'anger': 0.0, 'anticipation': 0.0, 'disgust': 0.0, 'fear': 0.33, 'joy': 0.0, 'sadness': 0.0, 'surprise': 0.33, 'trust': 0.33}, 'polarities': {'negative': 0.5, 'positive': 0.5}}, " Este documento é sobre a organização de um campo de treino'": {'emotions': {'anger': 0.0, 'anticipation': 0.25, 'disgust': 0.0, 'fear': 0.0, 'joy': 0.25, 'sadness': 0.0, 'surprise': 0.25, 'trust': 0.25}, 'polarities': {'negative': 0.0, 'positive': 1.0}}, " Assim sendo, o documento agora revelado não comprova a alegação de que o Governo da Ucrânia estaria 'sem sombra de dúvida' a preparar um ataque militar ao Donbass em março de 2022": {'emotions': {'anger': 0.22, 'anticipation': 0.11, 'disgust': 0.0, 'fear': 0.11, 'joy': 0.11, 'sadness': 0.11, 'surprise': 0.11, 'trust': 0.22}, 'polarities': {'negative': 0.5, 'positive': 0.5}}, "Putin foi apanhado a utilizar 'fund

{' Embora este tenha sido o segundo ano consecutivo de crescimento do país a este nível, os gastos militares reais da Rússia em 2020 foram 6,6% inferiores ao previsto no orçamento militar inicial': {'emotions': {'anger': 0.0, 'anticipation': 0.22, 'disgust': 0.11, 'fear': 0.0, 'joy': 0.0, 'sadness': 0.22, 'surprise': 0.11, 'trust': 0.33}, 'polarities': {'negative': 0.44, 'positive': 0.56}}, "'O aumento de 2,6% nos gastos militares mundiais aconteceu num ano em que o Produto Interno Bruto (PIB) global encolheu 4,4% (projeção de outubro de 2020 do Fundo Monetário Internacional), em grande parte devido aos impactos económicos da pandemia de Covid-19": {'emotions': {'anger': 0.0, 'anticipation': 0.17, 'disgust': 0.17, 'fear': 0.33, 'joy': 0.08, 'sadness': 0.25, 'surprise': 0.0, 'trust': 0.0}, 'polarities': {'negative': 0.7, 'positive': 0.3}}, 'Em Portugal, as despesas com a Defesa mantiveram-se relativamente estáveis ao longo dos últimos anos': {'emotions': {'anger': 0.0, 'anticipation': 0

{"'Suburbanos e Metropolitanos voltaram a encher nos últimos meses, mais pessoas que antes da pandemia": {'emotions': {'anger': 0.0, 'anticipation': 0.0, 'disgust': 0.0, 'fear': 0.5, 'joy': 0.0, 'sadness': 0.5, 'surprise': 0.0, 'trust': 0.0}, 'polarities': {'negative': 1.0, 'positive': 0.0}}, 'Na respetiva caixa de comentários há quem sugira outras hipóteses, nomeadamente a redução do número de comboios e carruagens a circular': {'emotions': {'anger': 0.0, 'anticipation': 0.25, 'disgust': 0.0, 'fear': 0.0, 'joy': 0.0, 'sadness': 0.0, 'surprise': 0.25, 'trust': 0.5}, 'polarities': {'negative': 0.0, 'positive': 0.0}}, '': {'emotions': {'anger': 0.0, 'anticipation': 0.0, 'disgust': 0.0, 'fear': 0.0, 'joy': 0.0, 'sadness': 0.0, 'surprise': 0.0, 'trust': 0.0}, 'polarities': {'negative': 0.0, 'positive': 0.0}}, " 'Em termos anuais, os resultados preliminares de 2021 indicam um aumento de 11,5% no número total de passageiros transportados por comboio face a 2020, após a redução significat

{' Ainda assim, está em causa a transferência de jogadores e a venda de bilhetes e merchandising': {'emotions': {'anger': 0.33, 'anticipation': 0.33, 'disgust': 0.0, 'fear': 0.0, 'joy': 0.0, 'sadness': 0.0, 'surprise': 0.33, 'trust': 0.0}, 'polarities': {'negative': 1.0, 'positive': 0.0}}, " Vamos ser implacáveis na perseguição daqueles que permitem a morte de civis, a destruição de hospitais e a ocupação ilegal de aliados soberanos', garantiu o primeiro-ministro britânico": {'emotions': {'anger': 0.21, 'anticipation': 0.03, 'disgust': 0.14, 'fear': 0.24, 'joy': 0.0, 'sadness': 0.24, 'surprise': 0.03, 'trust': 0.1}, 'polarities': {'negative': 0.75, 'positive': 0.25}}, " 'Com os laços estreitos que mantêm com Putin, eles são cúmplices da sua agressão": {'emotions': {'anger': 0.29, 'anticipation': 0.0, 'disgust': 0.0, 'fear': 0.43, 'joy': 0.0, 'sadness': 0.14, 'surprise': 0.0, 'trust': 0.14}, 'polarities': {'negative': 0.75, 'positive': 0.25}}, " Detentores de património que chega até ao

{" Pretende-se assim reafirmar o compromisso de defesa coletiva, exercitar a interoperabilidade e dissuadir possíveis ameaças'": {'emotions': {'anger': 0.25, 'anticipation': 0.25, 'disgust': 0.0, 'fear': 0.25, 'joy': 0.0, 'sadness': 0.25, 'surprise': 0.0, 'trust': 0.0}, 'polarities': {'negative': 0.67, 'positive': 0.33}}, 'Não é verdade que quatro caças F-16 de Portugal tenham sido mobilizados para a fronteira entre a Polónia e a Ucrânia, no contexto da guerra em curso no Leste da Europa': {'emotions': {'anger': 0.14, 'anticipation': 0.0, 'disgust': 0.0, 'fear': 0.29, 'joy': 0.14, 'sadness': 0.14, 'surprise': 0.0, 'trust': 0.29}, 'polarities': {'negative': 0.5, 'positive': 0.5}}, '': {'emotions': {'anger': 0.0, 'anticipation': 0.0, 'disgust': 0.0, 'fear': 0.0, 'joy': 0.0, 'sadness': 0.0, 'surprise': 0.0, 'trust': 0.0}, 'polarities': {'negative': 0.0, 'positive': 0.0}}, " 'Quatro aeronaves F16-M chegaram hoje à Polónia, completando, com os 70 militares da Força Aérea Portuguesa já no te

...

{' Portugal destaca-se (pela negativa) na cauda da Europa, em contraste com os países da Europa de Leste que fizeram parte integrante da União das Repúblicas Socialistas Soviéticas (URSS) ou do Pacto de Varsóvia': {'emotions': {'anger': 0.0, 'anticipation': 0.0, 'disgust': 0.0, 'fear': 0.0, 'joy': 0.0, 'sadness': 1.0, 'surprise': 0.0, 'trust': 0.0}, 'polarities': {'negative': 1.0, 'positive': 0.0}}, " Já depois de consumada a dissolução da União das Repúblicas Socialistas Soviéticas (URSS) em 1991, também a Checoslováquia (sob a órbita de Moscovo via Pacto de Varsóvia durante o período da 'Guerra Fria') avançou para um processo similar em 1993, dando origem a dois Estados independentes: a República Checa e a República Eslovaca (ou Eslováquia)": {'emotions': {'anger': 0.2, 'anticipation': 0.07, 'disgust': 0.0, 'fear': 0.27, 'joy': 0.07, 'sadness': 0.27, 'surprise': 0.13, 'trust': 0.0}, 'polarities': {'negative': 0.67, 'positive': 0.33}}, "A indicação 'Noticias NTD' corresponde à versão 

{'Apesar da dificuldade de analisar publicações em língua russa conseguimos identificar uma primeira partilha em 2015, na aplicação Coub': {'emotions': {'anger': 0.25, 'anticipation': 0.0, 'disgust': 0.0, 'fear': 0.25, 'joy': 0.0, 'sadness': 0.5, 'surprise': 0.0, 'trust': 0.0}, 'polarities': {'negative': 1.0, 'positive': 0.0}}, 'Importa porém ressalvar que não conseguimos apurar se essas descrições correspondem ao que realmente aconteceu, além de encontrarmos algumas versões dissonantes - nomeadamente que a detenção terá sido real e não encenada, por uma questão de direitos autorais da música, entre outros exemplos': {'emotions': {'anger': 0.0, 'anticipation': 0.33, 'disgust': 0.0, 'fear': 0.0, 'joy': 0.0, 'sadness': 0.33, 'surprise': 0.0, 'trust': 0.33}, 'polarities': {'negative': 0.4, 'positive': 0.6}}, 'Encontramos múltiplos exemplos, em diversos idiomas, todos apontando no mesmo sentido: as imagens mostram elementos do Coro do Exército de São Petersburgo, na Rússia, a serem violent

{' Infelizmente, este mês, o clássico de animação da 20th Century Studios, “Anastasia”, foi retirado da Disney+ nos EUA': {'emotions': {'anger': 0.0, 'anticipation': 0.0, 'disgust': 0.0, 'fear': 0.0, 'joy': 0.0, 'sadness': 1.0, 'surprise': 0.0, 'trust': 0.0}, 'polarities': {'negative': 0.33, 'positive': 0.67}}, ' No artigo é, contudo, referido que a retirada do filme foi uma surpresa porque a Disney+ não tem por hábito anunciar os títulos que vão sair do catálogo, ao contrário de outras plataformas como a Netflix e a Hulu': {'emotions': {'anger': 0.0, 'anticipation': 0.0, 'disgust': 0.0, 'fear': 0.0, 'joy': 0.0, 'sadness': 0.25, 'surprise': 0.5, 'trust': 0.25}, 'polarities': {'negative': 0.75, 'positive': 0.25}}, 'O filme é inspirado na lenda da princesa Anastasia, supostamente a única filha do czar Nicolau II que sobreviveu ao assassinato da sua família em 1918, após a Revolução Russa': {'emotions': {'anger': 0.17, 'anticipation': 0.08, 'disgust': 0.08, 'fear': 0.17, 'joy': 0.08, 'sad

{' Além de ter um nome diferente, como já sublinhámos': {'emotions': {'anger': 0.0, 'anticipation': 0.0, 'disgust': 0.0, 'fear': 0.0, 'joy': 0.0, 'sadness': 0.0, 'surprise': 1.0, 'trust': 0.0}, 'polarities': {'negative': 0.0, 'positive': 0.0}}, 'Por outro lado, na realidade, a primeira mulher piloto de combate da Força Aérea da Ucrânia chama-se Nadiya Savchenko, a qual foi presa em 2014 na Rússia, acusada de cumplicidade no assassinato de dois jornalistas russos no Leste da Ucrânia': {'emotions': {'anger': 0.21, 'anticipation': 0.0, 'disgust': 0.21, 'fear': 0.29, 'joy': 0.0, 'sadness': 0.21, 'surprise': 0.07, 'trust': 0.0}, 'polarities': {'negative': 0.71, 'positive': 0.29}}, 'Não encontramos qualquer registo público sobre a eventual morte de Savchenko': {'emotions': {'anger': 0.09, 'anticipation': 0.36, 'disgust': 0.09, 'fear': 0.18, 'joy': 0.0, 'sadness': 0.18, 'surprise': 0.09, 'trust': 0.0}, 'polarities': {'negative': 0.67, 'positive': 0.33}}, " 'Primeira piloto de caça morta na gu

{' O povo judeu em Israel é um povo único, uma população única': {'emotions': {'anger': 0.0, 'anticipation': 0.0, 'disgust': 0.0, 'fear': 0.0, 'joy': 0.0, 'sadness': 0.0, 'surprise': 1.0, 'trust': 0.0}, 'polarities': {'negative': 0.0, 'positive': 1.0}}, ' Pouco tempo antes da visita oficial a Israel (22 e 24 de janeiro), a Ucrânia saíra orgulhosamente do comité da Organização das Nações Unidas (ONU) sobre o Exercício dos Direitos do Povo Palestiniano, mais precisamente a 6 de janeiro': {'emotions': {'anger': 0.0, 'anticipation': 0.2, 'disgust': 0.0, 'fear': 0.0, 'joy': 0.0, 'sadness': 0.2, 'surprise': 0.2, 'trust': 0.4}, 'polarities': {'negative': 0.67, 'positive': 0.33}}, " No final, são os mesmos que reclamam de anti-semitismo'; 'Não, caros colegas jornalistas, Volodimyr Zelensky não é um herói e jamais o será": {'emotions': {'anger': 0.12, 'anticipation': 0.25, 'disgust': 0.0, 'fear': 0.0, 'joy': 0.12, 'sadness': 0.25, 'surprise': 0.12, 'trust': 0.12}, 'polarities': {'negative': 0.2